## The assignment asked that the following tasks be completed:


+     all of the attribute variables in __init__ (e.g., self.aligned) are filled by functions called during the .run() function.
+     the _run_raxml() function removes an existing output file with the same name if it exists.
+     the _run_raxml() function modifies the command for raxml by replacing the argument -n out with -n <outname>, and then running it with subprocess and parsing the result.
+     the _run_raxml() function parses the result from the raxml output file RAxML_bestTree.<outname> as a string and returns it.


### The data

In [5]:
fasta_data = """\
>sample1
AAGGCCTTAAGGCGTTAAAACCTTAAGGCCTTAAGGCCTT
>sample2
AAGGCCTTTTGGCGTTAAAACCTTAAGGCCTAAAGGCCTT
>sample3
AAGCCCTAAAGCCCTTAAGGCCAAGGCCATAAGGCCGTGG
>sample4
AAGGCCTAAAGGCCTTAAGGCCAAGGCCATAAGGCCGTCG
>sample5
TTGGCCTATAGGCCTTTAGGCCAAGGCCTTGACGCCTAAG
"""

### This was your starting point

In [1]:
class Phylogeny:
    def __init__(self, fasta_string):
        ## store data
        self.fasta = fasta_string
        self.aligned = None
        self.phylip = None
        self.tree = None
        self.log = None
        
    # private functions
    def _muscle_align(self):
        pass
    
    def _aligned_fasta_to_phylip(self):
        pass
    
    def _run_raxml(self, outname):
        pass
    
    # public function
    def run(self, outname):
        self.aligned = self._muscle_align()
        self.phylip = self._aligned_fasta_to_phylip()
        self.tree = self._run_raxml(outname)



### A completed Class object

In [118]:
import os
import shlex
import subprocess as sps


class Phylogeny:
    def __init__(self, fasta_string):
        ## store data
        self.fasta = fasta_string
        self.aligned = None
        self.phylip = None
        self.tree = None
        self.log = None
        
    # private functions
    def _muscle_align(self):
        "copied from the notebook. Stored results to log and aligned"
        proc = sps.Popen(
            ["muscle"],
            stdin=sps.PIPE,
            stdout=sps.PIPE, 
            stderr=sps.PIPE)
        stdout, stderr = proc.communicate(input=fasta_data.encode())
        self.log = [stderr]
        return stdout
    
    def _aligned_fasta_to_phylip(self):
        """
        copied from the notebook, writes output to 'aligned.phy'
        and returns as a string. Changed to use self.fasta
        """
        ntaxa = len(self.fasta.split(">")[1:])
        seqlen = len(self.fasta.split("\n")[1])
        seqs = [i.strip().replace("\n", "   ") for i in self.fasta.split(">")[1:]]
        seqstring = "\n".join(seqs)
        phylip = "{} {}\n{}".format(ntaxa, seqlen, seqstring)
        with open("aligned.phy", 'w') as out:
            out.write(phylip)
        return phylip
    
    def _run_raxml(self, outname):
        """
        copied from the notebook, changed -n to take outname argument
        and stored stderr to log. Then parsed tree file output to
        return as a string.
        """
        # remove old name
        oldfile = "RAxML_info.{}".format(outname)
        if os.path.exists(oldfile):
            os.remove(oldfile)
            
        # run tree inference
        cmd = ("raxmlHPC -f a -m GTRGAMMA -p 123 -x 123 -N 10 -s aligned.phy -n {}".format(outname))
        proc = sps.Popen(
            shlex.split(cmd), 
            stdout=sps.PIPE, 
            stderr=sps.PIPE)
        stdout, stderr = proc.communicate()
        self.log.append(stdout.decode())
        
        # parse result
        with open("RAxML_bestTree.{}".format(outname), 'r') as treef:
            treedata = treef.read()
        return treedata

    # public function
    def run(self, outname):
        self.aligned = self._muscle_align()
        self.phylip = self._aligned_fasta_to_phylip()
        self.tree = self._run_raxml(outname)



### Test the code

In [119]:
phy = Phylogeny(fasta_data)

In [120]:
phy.run('example')

In [121]:
phy.tree

'(sample2:0.01949487937064468476,(sample5:0.39284608930548642336,(sample3:0.05717847208080240745,sample4:0.03097573410056383980):0.00000100000050002909):2.93338593538679903716,sample1:0.05682838037625117383):0.0;\n'

In [122]:
import toytree
toytree.tree(phy.tree).draw();

sample2 sample1 sample5 sample3 sample4

### Automated checks on code

In [130]:
p = Phylogeny(fasta_data)
p.run(outname="code-review")

for attribute in [p.fasta, p.aligned, p.phylip, p.tree, p.log]:
    if attribute is None:
        print("the attribute {} should have been filled".format(attribute))

In [131]:
p = Phylogeny(fasta_data)
p.run(outname="code-review")

import os
os.path.exists("./RAxML_bestTree.code-review")

True

In [132]:
# open the bestTree file and compare it to self.tree as string data
with open("./RAxML_bestTree.code-review", 'r') as treedata:
    print(p.tree == treedata.read())

True
